In [15]:
from transformers import Trainer, BertTokenizer, BertForSequenceClassification
from datasets import load_from_disk
import optuna
import torch
import math
import base

In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [17]:
DATASET = "trec"

In [18]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits_fine")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits_fine")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits_fine")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented_fine")
tokenizer = BertTokenizer.from_pretrained("ndavid/autotrain-trec-fine-bert-739422530")

In [19]:
train = train_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the train dataset")
eval = eval_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the eval dataset")
test = test_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the test dataset")

train_aug = all_train_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the augmented dataset")

In [20]:
num_epochs = 30
batch_size = 128

In [21]:
#Nápočet epoch na steps
data_length = len(train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [22]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "adam_beta1" : trial.suggest_float("adam_beta1", 0.9, 0.99, step=0.01),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

In [23]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [24]:
def get_Bert():
    return BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=50)

In [25]:
base.reset_seed()

In [26]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bert-base_fine_hp-search", epochs=num_epochs, batch_size=batch_size)

In [27]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert(),
    #callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-base",
    n_trials=150
)

[I 2025-03-15 09:38:21,587] A new study created in memory with name: Test-base


Trial 0 with params: {'learning_rate': 1.0253509690168497e-05, 'weight_decay': 0.01, 'adam_beta1': 0.97, 'warmup_steps': 2}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.867300,3.826696,0.042163,0.008459,0.025886,0.006274
2,3.808300,3.777743,0.157654,0.010138,0.019207,0.009450
3,3.769800,3.734266,0.186984,0.033938,0.023454,0.011374
4,3.728500,3.698701,0.185151,0.011023,0.022636,0.010120
5,3.704000,3.666467,0.186984,0.016542,0.023014,0.010992
6,3.666800,3.635868,0.181485,0.017853,0.021370,0.008555
7,3.637900,3.607637,0.180568,0.019561,0.021096,0.008097
8,3.614800,3.580955,0.180568,0.019561,0.021096,0.008097
9,3.588400,3.555389,0.180568,0.023558,0.021096,0.008119
10,3.566300,3.530838,0.180568,0.023558,0.021096,0.008119


[I 2025-03-15 09:39:12,558] Trial 0 pruned. 


...

Trial 149 with params: {'learning_rate': 0.0004499563740490953, 'weight_decay': 0.0, 'adam_beta1': 0.93, 'warmup_steps': 0}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.349000,2.858869,0.409716,0.072132,0.093801,0.073905
2,2.552300,2.215647,0.540788,0.168980,0.178683,0.160364
3,1.958800,1.747913,0.640697,0.273792,0.261621,0.246225
4,1.507600,1.473900,0.703025,0.306409,0.330714,0.298926
5,1.186200,1.305043,0.722273,0.315198,0.355448,0.325776
6,0.945500,1.175576,0.748854,0.464182,0.414157,0.406327
7,0.756600,1.124920,0.755270,0.454972,0.432995,0.428937
8,0.633700,1.074999,0.768103,0.481145,0.451322,0.446282
9,0.519800,1.043069,0.764436,0.486491,0.476648,0.468342
10,0.433500,1.024741,0.767186,0.459329,0.461784,0.453779


[W 2025-03-15 14:18:42,874] Trial 149 failed with parameters: {'learning_rate': 0.0004499563740490953, 'weight_decay': 0.0, 'adam_beta1': 0.93, 'warmup_steps': 0} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations/integration_utils.py", line 250, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2241, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2553, in _inner_training_loop
    and (torch.isnan(tr_loss_step) or torch.isinf(tr_loss_step))
KeyboardInterrupt
[W 2025-03-15 14:18:42,878] Trial 149 failed with value None.


KeyboardInterrupt: 

In [29]:
print(best_trial)

BestRun(run_id='118', objective=0.7156145737360871, hyperparameters={'learning_rate': 0.0004762367229644209, 'weight_decay': 0.002, 'adam_beta1': 0.93, 'warmup_steps': 0}, run_summary=None)


In [36]:
base.reset_seed()

In [37]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bert-distill_fine_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [38]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "adam_beta1" : trial.suggest_float("adam_beta1", 0.9, 0.99, step=0.01),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [39]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [40]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert(),
    #callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
best_trial2 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-destilace",
    n_trials=150
)

[I 2025-03-15 14:20:09,293] A new study created in memory with name: Test-destilace


Trial 0 with params: {'learning_rate': 1.0253509690168497e-05, 'weight_decay': 0.01, 'adam_beta1': 0.97, 'warmup_steps': 2, 'lambda_param': 0.1, 'temperature': 2.5}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.459400,2.429851,0.041247,0.008160,0.026134,0.006414
2,2.423900,2.396845,0.160403,0.009651,0.019518,0.009362
3,2.396100,2.368189,0.189734,0.034879,0.024276,0.012312
4,2.371700,2.344594,0.184235,0.011102,0.022362,0.009811
5,2.349700,2.323391,0.183318,0.014357,0.021918,0.009339


[I 2025-03-15 14:20:35,601] Trial 0 pruned. 


...

[I 2025-03-15 18:35:57,531] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 8.049073025334763e-05, 'weight_decay': 0.008, 'adam_beta1': 0.9, 'warmup_steps': 2, 'lambda_param': 0.2, 'temperature': 4.0}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.378100,2.268718,0.176902,0.003538,0.020000,0.006012
2,2.216100,2.123328,0.241980,0.074620,0.038501,0.033579
3,2.089600,1.994844,0.401467,0.054775,0.085465,0.063619
4,1.962700,1.873021,0.429881,0.090485,0.099238,0.077283
5,1.854300,1.760185,0.467461,0.105408,0.123442,0.100630
6,1.740400,1.663596,0.494959,0.121527,0.138103,0.112556
7,1.644700,1.582850,0.505041,0.147642,0.147991,0.126248
8,1.568500,1.507406,0.553621,0.222359,0.181137,0.164422
9,1.488200,1.438184,0.565536,0.242104,0.191957,0.176919
10,1.417500,1.382984,0.599450,0.241767,0.218564,0.203927


[I 2025-03-15 18:37:58,916] Trial 149 pruned. 


In [42]:
print(best_trial2)

BestRun(run_id='80', objective=0.6434396489980481, hyperparameters={'learning_rate': 0.000482322168974171, 'weight_decay': 0.006, 'adam_beta1': 0.92, 'warmup_steps': 2, 'lambda_param': 0.0, 'temperature': 2.5}, run_summary=None)


In [43]:
base.reset_seed()

In [44]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base_fine_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bert-base_fine_aug_hp-search", epochs=num_epochs, batch_size=batch_size)

In [45]:
#Nápočet epoch na steps
data_length = len(all_train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [46]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "adam_beta1" : trial.suggest_float("adam_beta1", 0.9, 0.99, step=0.01),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [47]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [48]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert(),
    #callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
best_trial3 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-base-aug",
    n_trials=150
)

[I 2025-03-15 18:38:00,160] A new study created in memory with name: Test-base-aug


Trial 0 with params: {'learning_rate': 1.0253509690168497e-05, 'weight_decay': 0.01, 'adam_beta1': 0.97, 'warmup_steps': 32}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.586900,3.324262,0.327223,0.043095,0.065415,0.047883
2,3.091700,2.904099,0.439047,0.109918,0.110082,0.094993
3,2.703900,2.574500,0.497709,0.129979,0.142067,0.118518
4,2.389700,2.308230,0.549954,0.215766,0.182629,0.165596
5,2.125800,2.098330,0.593951,0.261190,0.223329,0.206112


[I 2025-03-15 18:40:04,534] Trial 0 pruned. 


...

Trial 145 with params: {'learning_rate': 0.00014074166190486731, 'weight_decay': 0.007, 'adam_beta1': 0.91, 'warmup_steps': 38}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.972100,1.222858,0.748854,0.384880,0.405142,0.381300
2,0.591200,1.017145,0.770852,0.489351,0.493473,0.484553
3,0.263500,0.991532,0.788268,0.651248,0.595188,0.606448
4,0.134300,1.044246,0.792851,0.717364,0.626150,0.650823
5,0.078300,1.075532,0.798350,0.767169,0.691195,0.712200
6,0.051600,1.146067,0.782768,0.774485,0.662996,0.695900
7,0.034800,1.216859,0.786434,0.806122,0.687725,0.716664
8,0.026600,1.217579,0.796517,0.783812,0.719046,0.731645
9,0.021900,1.286508,0.787351,0.787690,0.688259,0.716363
10,0.015900,1.349379,0.784601,0.801483,0.707703,0.729129


Exception in thread Thread-24023 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 59, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 35, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
[W 2025-03-16 19:18:09,168] Trial 145 failed with parameters: {'learning_rate': 0.00014074166190486731, 'weight_decay': 0.007, 'adam_beta1': 0.91, 'warmup_steps': 38} because of the following error: KeyboardInterr

KeyboardInterrupt: 

In [51]:
print(best_trial3)

NameError: name 'best_trial3' is not defined

In [ ]:
base.reset_seed()

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill_fine_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bert-distill_fine_aug_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [ ]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "adam_beta1" : trial.suggest_float("adam_beta1", 0.9, 0.99, step=0.01),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [ ]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert(),
    #callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)
  

In [ ]:
best_trial4 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-Distill-aug",
    n_trials=150
)

In [ ]:
print(best_trial4)

BestRun(run_id='136', objective=0.7874461791210883, hyperparameters={'learning_rate': 0.0021806066601338593, 'weight_decay': 0.001, 'adam_beta1': 0.9, 'warmup_steps': 50, 'lambda_param': 0.8, 'temperature': 4.0}, run_summary=None)


In [ ]:
print("Best normal training score: ", best_trial)
print("Best distilation trianing score: ", best_trial2)
print("Best normal training score with augmentations: ", best_trial3)
print("Best distilation trianing score with augmentations: ",best_trial4)

Best normal training score:  BestRun(run_id='41', objective=0.7157156862853267, hyperparameters={'learning_rate': 0.004873101422020569, 'weight_decay': 0.001, 'adam_beta1': 0.93, 'warmup_steps': 4}, run_summary=None)
Best distilation trianing score:  BestRun(run_id='115', objective=0.7423955520558099, hyperparameters={'learning_rate': 0.004092058596290564, 'weight_decay': 0.008, 'adam_beta1': 0.92, 'warmup_steps': 2, 'lambda_param': 1.0, 'temperature': 4.0}, run_summary=None)
Best normal training score with augmentations:  BestRun(run_id='69', objective=0.7829410034428768, hyperparameters={'learning_rate': 0.0014622771684147115, 'weight_decay': 0.006, 'adam_beta1': 0.93, 'warmup_steps': 46}, run_summary=None)
Best distilation trianing score with augmentations:  BestRun(run_id='136', objective=0.7874461791210883, hyperparameters={'learning_rate': 0.0021806066601338593, 'weight_decay': 0.001, 'adam_beta1': 0.9, 'warmup_steps': 50, 'lambda_param': 0.8, 'temperature': 4.0}, run_summary=Non